In [ ]:
# 自定义类型的行为可以像内置类型那样自然（使用各种函数和方法），考的不是继承，而是鸭子类型（duck typing）：
# 按照预定行为实现对象所需的方法即可。


# 获取对象的字符串表示形式
# repr() 依赖 __repr__方法，以便于开发者理解的方式返回对象的字符串表示形式
# str() 依赖 __str__方法，以便于用户理解的方式返回对象的字符串表示形式

# bytes() 依赖__bytes__，返回对象的字节序列表示形式
# format()依赖__format__（也会被str.format方法调用），返回对象的特殊格式字符串表示形式

from array import array
import math


class Vector2d:
    typecode = 'd'  # 类属性，在实例和字节序列间转换时使用 

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # 实现可迭代，才能拆包。也可以写成 yield self.x; yield.self.y

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # {!r}获取各个分量的表示形式，然后插值构成字符串

    def __str__(self):
        return str(tuple(self))  # 显示为一个元组

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # typecode转成字节序列
                bytes(array(self.typecode, self)))  # 迭代实例得到数组再转成序列

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # 快速比较

    def __abs__(self):
        return math.hypot(self.x, self.y)  # 取模

    def __bool__(self):
        return bool(abs(self))  # 按模判断
    
    @classmethod  # 通常用来实现备选构造方法  定义操作类的方法而不是操作实例的方法
    def frombytes(cls, octets):  # 第一个参数是类本身，注意return
        typecode = chr(octets[0])  # 取出typecode
        memv = memoryview(octets[1:]).cast(typecode)  # 创建后使用typecode转换
        return cls(*memv)  # 拆包转换后的memoryview，得到缺省构造方法所需参数
    
v1 = Vector2d(3, 4)
print(v1.x, v1.y)
x, y = v1
print(x, y)
v1_clone = eval(repr(v1))
print(v1 == v1_clone)
print(v1)
octets = bytes(v1)
print(octets)
print(abs(v1))
print(bool(v1), bool(Vector2d(0, 0)))

v2_clone = Vector2d.frombytes(bytes(v1))
print(v1_clone == v2_clone)
print(v2_clone)

In [ ]:
# 如果修改实例的类属性，只能影响该实例(新建实例属性，而不影响类属性)。想修改所有所有实例的类属性，则要要用类名称
class cls:
    cls_proper = 'value1'

o1 = cls()
print(o1.cls_proper)
o2 = cls()
o2.cls_proper = 'value2'
print(o1.cls_proper)
cls.cls_proper = 'value3'
print(o1.cls_proper)
print(o2.cls_proper)

In [ ]:
# format(my_obj, format_spec) 第二个参数format_spec是格式说明符
# str.format()

# 格式说明符使用的表示法叫格式规范微语言
# str.format() 方法使用的是 格式字符串句法 {:}代换字段表示法，包括标准转换标志 !s、!r、和!a

In [ ]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)  # 给私有属性__x __y赋值
        self.__y = float(y)

    @property  # @property装饰器把方法标记为属性
    def x(self):  # 方法与属性同名
        return self.__x  # 访问私有属性

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))  # 迭代对象实现

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):    # 可散列对象实现
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):   # 格式说明符以p结尾则使用极坐标输出，利用angle方法计算
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>' # 使用<>区分极坐标与()直角坐标
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):  # 备用构造函数，实现从字节序列转换
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

v1 = Vector2d(3, 4)
print(v1.x, v1.y)
x, y = v1
print(x, y)
v1_clone = eval(repr(v1))
print(v1 == v1_clone)
print(v1)
octets = bytes(v1)
print(octets)
print(abs(v1))
print(bool(v1), bool(Vector2d(0, 0)))

v2_clone = Vector2d.frombytes(bytes(v1))
print(v1_clone == v2_clone)
print(v2_clone)
print(format(v1))
print(format(v1, '.2f'))
print(format(v1, '.3e'))
print(format(Vector2d(1, 1), '0.5fp'))
print(hash(v1_clone), hash(v2_clone))
print(len([v1_clone, v2_clone]))
print(v1.x, v1.y)

# 只读属性，私有属性被自动进行了名称改写name mangling
print(v1.__dict__)
v1.x = 123 # 失败

In [ ]:
# __slot__ 
# 默认情况下Python在各个实例中名为__dict__的字典里存储实例属性（如上面的_Vector2d__x），为提升访问速度而消耗了大量内存
# 如果要处理百万级别属性不多的实例，可以通过__slot__类属性，而不用字典。
# 如果要处理数百万个数值对象，应该使用NumPy数组，能高效使用内存。
# 继承自超类的__slot__属性没有效果，Python只会使用各个类中定义的__slot__属性。

class Vector2d:
    __slots__ = ('__x', '__y')   # 使用__slot__属性的元组，避免使用字典

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

v1 = Vector2d(3, 4)
print(v1.x, v1.y)
x, y = v1
print(x, y)
v1_clone = eval(repr(v1))
print(v1 == v1_clone)
print(v1)
octets = bytes(v1)
print(octets)
print(abs(v1))
print(bool(v1), bool(Vector2d(0, 0)))

v2_clone = Vector2d.frombytes(bytes(v1))
print(v1_clone == v2_clone)
print(v2_clone)
print(format(v1))
print(format(v1, '.2f'))
print(format(v1, '.3e'))
print(format(Vector2d(1, 1), '0.5fp'))
print(hash(v1_clone), hash(v2_clone))
print(len([v1_clone, v2_clone]))
print(v1.x, v1.y)

print(v1.__slots__)
print(v1.__dict__) # 不存在
